In [1]:
import pandas as pd
import numpy as np
import os

# Keys

In [2]:
GROQ_API_KEY = os.environ.get("GROQ")
GOOGLE_API_KEY = os.environ.get("GOOGLE")

# Basic Example

In [3]:
from groq import Groq

client = Groq(
    api_key=GROQ_API_KEY,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of low latency LLMs",
        }
    ],
    model="mixtral-8x7b-32768",
)

print(chat_completion.choices[0].message.content)

ModuleNotFoundError: No module named 'groq'

# Via Langchain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

# Client
chat = ChatGroq(temperature=0, groq_api_key=GROQ_API_KEY, model_name="mixtral-8x7b-32768")

# Setup
# Write a prompt and invoke ChatGroq to create completions:
system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
response = chain.invoke({"text": "Explain the importance of low latency LLMs."})
response

# Via Langchain + System

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

# Client
chat = ChatGroq(temperature=0, groq_api_key=GROQ_API_KEY, model_name="mixtral-8x7b-32768")

# Setup
# Write a prompt and invoke ChatGroq to create completions:
system = "You are a helpful assistant that re-writes the user's text to sound more upbeat"

human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
response = chain.invoke({"text": "I am tired"})
response

AIMessage(content="You're running on reserves but soon you'll get a chance to recharge!\n\n(I'm here to make your text more positive and engaging!)\n\nLet me know if you need help with that or anything else!")

# Embedding
https://python.langchain.com/docs/integrations/text_embedding/google_generative_ai

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key = GOOGLE_API_KEY)
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.05636945, 0.0048285457, -0.0762591, -0.023642512, 0.05329321]

In [7]:
vectors = embeddings.embed_documents(
    [
        "Today is Monday",
        "Today is Tuesday",
        "Today is April Fools day",
    ]
)
len(vectors), len(vectors[0])

(3, 768)

In [14]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("assets/fwd_big3.md").load()



[Document(page_content="# FWD Big 3: Comprehensive Protection for What Matters Most\nDescription: In today's world, unexpected health events can have a significant financial impact. FWD's Big 3 Critical Illness insurance plan offers affordable and convenient protection against three of the most common critical illnesses: cancer, heart attack, and stroke.\n\n## Benefits:\n\n- One-time lump sum payout: Upon diagnosis of any covered critical illness, you'll receive a 100% cash payout of your chosen sum insured (up to S$200,000). This allows you to focus on your recovery without financial worry.\n- Flexible coverage options: Choose from three coverage amounts (S$50,000, S$100,000, or S$200,000) to suit your needs and budget.\n- No medical examination required: Get covered quickly and easily by answering a simple health declaration. This eliminates the need for a medical exam, making the application process fast and convenient.\n- Bridge the protection gap: This plan can be used as a standa

# Chroma
incomplete

In [15]:
from langchain.vectorstores import Chroma

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key = GOOGLE_API_KEY)

docs = loader
# Save to disk
vectorstore = Chroma.from_documents(
                     documents=docs,                 # Data
                     embedding=gemini_embeddings,    # Embedding model
                    #  persist_directory="./chroma_db" # Directory to save data
                     )
# Create a retriever using Chroma
# Load from disk
vectorstore_disk = Chroma(
                    #     persist_directory="./chroma_db",       # Directory of db
                        embedding_function=gemini_embeddings   # Embedding model
                   )
# 1. If you don't pass this value, you will get a warning.
retriever = vectorstore_disk.as_retriever( search_kwargs={"k": 15}) #" "score_threshold": .3 search_type ='similarity_score_threshold',

# Check if the retriever is working by trying to fetch the relevant docs related
# to the word 'climate'. If the length is greater than zero, it means that
# the retriever is functioning well.
print(len(retriever.get_relevant_documents("claims for critical illness")))

RuntimeError: [91mYour system has an unsupported version of sqlite3. Chroma                     requires sqlite3 >= 3.35.0.[0m
[94mPlease visit                     https://docs.trychroma.com/troubleshooting#sqlite to learn how                     to upgrade.[0m

### Langchain + Documents Loader + RAG
incomplete

In [1]:

# from langchain_community.vectorstores import DocArrayInMemorySearch
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import RunnableParallel, RunnablePassthrough
# # from langchain_openai.chat_models import ChatOpenAI
# from langchain_openai.embeddings import OpenAIEmbeddings

# vectorstore = DocArrayInMemorySearch.from_texts(
#     ["harrison worked at kensho", "bears like to eat honey"],
#     embedding=OpenAIEmbeddings(),
# )
# retriever = vectorstore.as_retriever()

# template = """Answer the question based only on the following context:
# {context}

# Question: {question}
# """
# prompt = ChatPromptTemplate.from_template(template)
# model = ChatGroq(temperature=0, groq_api_key=GROQ_API_KEY, model_name="mixtral-8x7b-32768")
# output_parser = StrOutputParser()

# setup_and_retrieval = RunnableParallel(
#     {"context": retriever, "question": RunnablePassthrough()}
# )
# chain = setup_and_retrieval | prompt | model | output_parser

# chain.invoke("where did harrison work?")

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [12]:
# loader = JSONLoader(
#     file_path='assets/dune_characters.json',
#     jq_schema='',
#     text_content=False)

# data = loader.load()

In [13]:
file_path='assets/dune_characters.json'
data = json.loads(Path(file_path).read_text())

In [14]:
data

[{'name': 'Paul Atreides',
  'description': 'The young protagonist and heir of House Atreides. He is trained in combat, strategy, and possesses unique mental abilities. Paul is a complex and conflicted character, wrestling with his destiny and the potential for both heroism and tyranny.',
  'character': 'Hero, Messiah',
  'persona': 'Reserved, contemplative, burdened by responsibility'},
 {'name': 'Lady Jessica',
  'description': "Paul's mother, a member of the Bene Gesserit sisterhood with advanced mental and physical training. Lady Jessica is a strong and intelligent woman, torn between her loyalty to her son and the Bene Gesserit.",
  'character': 'Bene Gesserit Reverend Mother, mother',
  'persona': 'Stoic, cunning, protective'},
 {'name': 'Duke Leto Atreides',
  'description': "Paul's father, a noble and just leader of House Atreides entrusted with governing Arrakis. Duke Leto is a wise and compassionate leader, respected for his fairness and integrity.",
  'character': 'Noble Duk

In [8]:
markdown_path = "assets/fwd_big3.md"
loader = UnstructuredMarkdownLoader(markdown_path).load()

ModuleNotFoundError: No module named 'markdown'